In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

In [2]:
def calc_metrics(dataset, labels, header):
    pred_labels=[l+'_pred' for l in labels]
    y_true = dataset[labels]
    y_pred = dataset[pred_labels]
    metrics = precision_recall_fscore_support(y_true, y_pred)
    metrics = [list(m) for m in metrics]

    avg = precision_recall_fscore_support(y_true, y_pred, average='macro')
    avg = np.array(avg)
    avg[3] = np.sum(metrics[3])

    m = np.array(metrics)
    m2 = np.append(m.T,  np.array(avg).reshape(1,4), axis=0)
    m2=m2[:,:3]
    stat=m2.flatten()
    stat=stat.reshape(-1,1).T * 100
    lstat=stat.flatten().tolist()
    header.extend(lstat)
    return header

# Models

In [3]:
path='../results/predictions_taska.csv'
ds = pd.read_csv(path,keep_default_na=False)
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66200 entries, 0 to 66199
Data columns (total 8 columns):
subtask_a         66200 non-null object
subtask_a_pred    66200 non-null object
run_id            66200 non-null object
timestamp         66200 non-null float64
model             66200 non-null object
set               66200 non-null object
kfold             66200 non-null int64
id                66200 non-null int64
dtypes: float64(1), int64(2), object(5)
memory usage: 4.0+ MB


In [4]:
ds.subtask_a.unique(),ds.subtask_a_pred.unique()

(array(['NOT', 'OFF'], dtype=object), array(['NOT', 'OFF'], dtype=object))

In [5]:
ds.groupby(['model','run_id','set']).agg({'subtask_a_pred': 'count'})

subtask_a_pred
model    run_id                               set                 
bilstm   e4a43c79-d251-46e2-b431-c0115b70a42f cv             10592
                                              test            2648
cnn      031f4071-a31c-42ed-970e-e2c4056efb73 cv             10592
                                              test            2648
fasttext 965a1b10-32a6-4474-85af-b9b9444a00c4 cv             10592
                                              test            2648
lr       beff7b70-dd2a-4728-acbf-855b8e3afd35 cv             10592
                                              test            2648
lstm     d4331456-4c3b-4e5f-954e-d751a61e862a cv             10592
                                              test            2648

In [6]:
results = ds[(ds.model=='cnn') & (ds.set=='test')]
print(classification_report(results.subtask_a, results.subtask_a_pred))

              precision    recall  f1-score   support

         NOT       0.83      0.75      0.79      1759
         OFF       0.59      0.69      0.63       889

   micro avg       0.73      0.73      0.73      2648
   macro avg       0.71      0.72      0.71      2648
weighted avg       0.75      0.73      0.74      2648



In [7]:
benchmarking = []

labels =['subtask_a']

results = ds[(ds.model=='lr') & (ds.set=='test')]
m=calc_metrics(results, labels, ['LR'])
benchmarking.append(m)

results = ds[(ds.model=='cnn') & (ds.set=='test')]
m=calc_metrics(results, labels, ['CNN'])
benchmarking.append(m)

results = ds[(ds.model=='lstm') & (ds.set=='test')]
m=calc_metrics(results, labels, ['LSTM'])
benchmarking.append(m)

results = ds[(ds.model=='bilstm') & (ds.set=='test')]
m=calc_metrics(results, labels, ['BI-LSTM'])
benchmarking.append(m)

results = ds[(ds.model=='fasttext') & (ds.set=='test')]
m=calc_metrics(results, labels, ['FastText'])
benchmarking.append(m)


#benchmarking

In [8]:
c = ['model', 'NOT-P', 'NOT-R ', 'NOT-F1','OFF-P', 'OFF-R', 'OFF-F1','P', 'R', 'F1']
dsstat=pd.DataFrame(benchmarking, columns=c)
path = '../results/models_performance.csv'
dsstat.to_csv(path, index=False)
dsstat

,model,NOT-P,NOT-R,NOT-F1,OFF-P,OFF-R,OFF-F1,P,R,F1
0,LR,81.801471,75.895395,78.737835,58.267717,66.591676,62.152231,70.034594,71.243536,70.445033
1,CNN,82.894737,75.213189,78.867362,58.555133,69.291339,63.472437,70.724935,72.252264,71.169900
2,LSTM,78.034047,80.784537,79.385475,59.129383,55.005624,56.993007,68.581715,67.895080,68.189241
3,BI-LSTM,79.549114,84.252416,81.833241,64.713376,57.142857,60.692951,72.131245,70.697637,71.263096
4,FastText,77.871148,79.022172,78.442438,57.242178,55.568054,56.392694,67.556663,67.295113,67.417566


# Embeddings

In [23]:
path='../results/predictions_embeddings_random.csv'
ds = pd.read_csv(path,keep_default_na=False)
ds['model'] = 'cnn'

path='../results/predictions_embeddings_word2vec.csv'
tmp = pd.read_csv(path,keep_default_na=False)
tmp['model'] = 'cnn-w2v'
ds = ds.append(tmp)

path='../results/predictions_embeddings_fasttext.csv'
tmp = pd.read_csv(path,keep_default_na=False)
tmp['model'] = 'cnn-fasttext'
ds = ds.append(tmp)

ds.shape

(39720, 8)

In [22]:
ds.loc[:13239,:].to_csv('../results/predictions_embeddings_random.csv', index=False)
ds.loc[13240:26479,:].to_csv('../results/predictions_embeddings_word2vec.csv', index=False)
ds.loc[26480:39719,:].to_csv('../results/predictions_embeddings_fasttext.csv', index=False)

In [24]:
ds.groupby(['model','run_id','set']).agg({'subtask_a_pred': 'count'})

subtask_a_pred
model        run_id                               set                 
cnn          a7e54bea-6fe0-4d0d-a964-b97e4205d5ee cv             10592
                                                  test            2648
cnn-fasttext ed943f4f-7e49-4928-8e1c-c8aae6d32500 cv             10592
                                                  test            2648
cnn-w2v      53ad593b-aa67-4720-9a5b-484004173c2d cv             10592
                                                  test            2648

In [29]:
results = ds[(ds.model=='cnn') & (ds.set=='test')]
print(classification_report(results.subtask_a, results.subtask_a_pred))

              precision    recall  f1-score   support

         NOT       0.79      0.83      0.81      1759
         OFF       0.63      0.57      0.60       889

   micro avg       0.74      0.74      0.74      2648
   macro avg       0.71      0.70      0.71      2648
weighted avg       0.74      0.74      0.74      2648



In [28]:
benchmarking = []

labels =['subtask_a']

results = ds[(ds.model=='cnn') & (ds.set=='test')]
m=calc_metrics(results, labels, ['Random'])
benchmarking.append(m)

results = ds[(ds.model=='cnn-w2v') & (ds.set=='test')]
m=calc_metrics(results, labels, ['Word2Vec'])
benchmarking.append(m)

results = ds[(ds.model=='cnn-fasttext') & (ds.set=='test')]
m=calc_metrics(results, labels, ['Fasttext'])
benchmarking.append(m)


In [27]:
c = ['model', 'NOT-P', 'NOT-R ', 'NOT-F1','OFF-P', 'OFF-R', 'OFF-F1','P', 'R', 'F1']
dsstat=pd.DataFrame(benchmarking, columns=c)
path = '../results/models_embeddings.csv'
dsstat.to_csv(path, index=False)
dsstat

,model,NOT-P,NOT-R,NOT-F1,OFF-P,OFF-R,OFF-F1,P,R,F1
0,Random,79.379761,82.944855,81.123158,62.962963,57.367829,60.035315,71.171362,70.156342,70.579237
1,Word2Vec,78.629250,82.831154,80.675526,62.012579,55.455568,58.551069,70.320914,69.143361,69.613297
2,Fasttext,82.543491,78.226265,80.326912,60.958206,67.266592,63.957219,71.750849,72.746428,72.142066
